In [1]:
# How to Run Django in Jupyter Notebook
# https://www.youtube.com/watch?v=TN5zIA--cz4&ab_channel=TechSunami

In [2]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from django.db import connection
from django.db import reset_queries
from django.db import models
from django_orm_app.models import Author, Blog

In [42]:
def define_model(model):
    print(f"----------------- {model.__name__}-------------------")
    for field in model._meta.fields:
        print(f"{field.name} --> {field.get_internal_type()}")
    print('\033[1m' + f"Table name in database is {model._meta.db_table}" + '\033[0m')

In [43]:
define_model(Author)
define_model(Blog)

----------------- Author-------------------
id --> BigAutoField
name --> CharField
email --> CharField
age --> IntegerField
creation_date --> DateTimeField
updation_date --> DateTimeField
Table name in database is author_model
----------------- Blog-------------------
id --> BigAutoField
name --> CharField
author --> ForeignKey
tagline --> TextField
creation_date --> DateTimeField
updation_date --> DateTimeField
Table name in database is blog_model


## Table of Content
1. [Model Creation Type](#1)
2. [Understand Query Evaluation](#2)
    1. [Queryset are Lazy](#2.a) 
    2. [How Queryset held data in Memory](#2.b)
3. [Method that don't return Queryset](#3)
    1. [get](#3.a) 
    2. [get_or_create](#3.b) 
    3. [update_or_create ](#3.c) 
    4. [bulk_create](#3.d) 
    5. [bulk_update](#3.e) 
    6. [count](#3.f) 
    7. [latest](#3.g) 
    8. [earliest](#3.h) 
    9. [last](#3.i) 
    10. [first](#3.j) 
    11. [exists](#3.k) 
    12. [contains](#3.l)
    13. [delete](#3.m)
    14. [update](#3.n)
    15. [aggregate](#3.o)

In [ ]:
def get_query():
    return connection.queries

<a id="1"></a>
## Model Creation Type


In [ ]:
author_instance = Author(
    name = "Lakshit", 
    email="lakshit@gmail.com"
)
author_instance.save()

In [ ]:
blog_instance = Blog.objects.create(
    name="blog no 1", 
    tagline="All the latest bloggggg news.",
    author=author_instance
)

In [ ]:
blog_dict = {
    "name":"Blog number 2", 
    "tagline":"This is best blog everrrrrrrrr......",
    "author":author_instance
}
blog_instance_2 = Blog.objects.create(**blog_dict)

In [ ]:
get_query()

In [ ]:
reset_queries()

A <b>QuerySet</b> represents a collection of objects from your database. It can have zero, one or many filters. Filters narrow down the query results based on the given parameters. In SQL terms, a QuerySet equates to a SELECT statement, and a filter is a limiting clause such as WHERE or LIMIT .

<a id="2"></a>
## Understand Query Evaluation

<a id="2.a"></a>
### Queryset are Lazy
<pre>QuerySets are lazy – the act of creating a QuerySet doesn’t involve any database activity. You can stack filters together all day long, and Django won’t actually run the query until the QuerySet is evaluated. Take a look  at below example:</pre>

Django Documentation - https://docs.djangoproject.com/en/4.2/topics/db/queries/#querysets-are-lazy

In [ ]:
q = Blog.objects.filter(id = 1)

In [ ]:
get_query()

In [ ]:
reset_queries()

<a id="2.b"></a>
### How Queryset held data in Memory

Django Documentation - https://docs.djangoproject.com/en/4.2/topics/db/queries/#caching-and-querysets

In [ ]:
q = Blog.objects.all()

In [ ]:
q

In [ ]:
get_query()

In [ ]:
print([p.name for p in q])

In [ ]:
Blog.objects.filter(id = 1).update(name = "laskhit")

In [ ]:
q[0].__dict__

In [ ]:
q[0].name

<a id="3"></a>
## Method that don't return Queryset
Django Documentation - https://docs.djangoproject.com/en/4.2/ref/models/querysets/#methods-that-do-not-return-querysets

<a id="3.a"></a>
### get

Django Documentation - https://docs.djangoproject.com/en/4.2/ref/models/querysets/#get

In [ ]:
obj = Blog.objects.get(id = 1)

In [ ]:
try:
    obj = Blog.objects.get(id = 100)
except Blog.DoesNotExist:
    print("In Does Not exist Block")
except Blog.MultipleObjectsReturned:
    print("In Multi Object Block")

<a id="3.b"></a>
### get_or_create

Django Documentation - https://docs.djangoproject.com/en/4.2/ref/models/querysets/#get-or-create

In [ ]:
#Traditional Way
try:
    obj = Blog.objects.get(id = 100)
except Blog.DoesNotExist:
    blog_dict = {
        "name":"Blog number 2", 
        "tagline":"This is best blog everrrrrrrrr......"
    }
    obj = Blog.objects.create(**blog_dict)

In [ ]:
obj, created = Blog.objects.get_or_create(
    id=100,
    defaults={
        "name":"Blog number 2", 
        "tagline":"This is best blog everrrrrrrrr......"
    },
)

<a id="3.c"></a>
### update_or_create 
Django Documentation - https://docs.djangoproject.com/en/4.2/ref/models/querysets/#update-or-create

In [ ]:
search_field = {"name": "Lakshit"}
update_field = {"name": "Techsunami"}
try:
    obj = Author.objects.get(**search_field)
    for key, value in update_field.items():
        setattr(obj, key, value)
    obj.save()
except Author.DoesNotExist:
    new_values = search_field
    new_values.update(update_field)
    obj = Author(**new_values)
    obj.save()

In [ ]:
obj

In [ ]:
obj, created = Author.objects.update_or_create(
    name="Lakshit",
    defaults={"name": "Techsunami"},
)

<a id="3.d"></a>
### bulk_create

<pre>
<b>Note</b>
 - The model’s save() method will not be called, and the pre_save and post_save signals will not be sent.
</pre>

Django Documentation - https://docs.djangoproject.com/en/4.2/ref/models/querysets/#bulk-create/

In [ ]:
reset_queries()

In [ ]:
get_query()

In [ ]:
objs = Blog.objects.bulk_create(
    [
        Blog(name="Blog number 3"),
        Blog(name="Blog number 4"),
        Blog(name="Blog number 5")
    ]
)

In [ ]:
get_query()

<a id="3.e"></a>
### bulk_update

Django Documentation - https://docs.djangoproject.com/en/4.2/ref/models/querysets/#bulk-update

In [ ]:
objs[0].name = "SOmething 1"
objs[1].name = "something 2"
objs[2].name = "something 3"

Blog.objects.bulk_update(objs, ["name"])

In [ ]:
objs

In [ ]:
get_query()

<a id="3.f"></a>
### count

Django Documentation - https://docs.djangoproject.com/en/4.2/ref/models/querysets/#count

In [ ]:
objs = Blog.objects.all()

In [ ]:
# Not optimized
len(objs)
print(f"Query made is {get_query()[-1]}")

In [ ]:
# optimized
Blog.objects.count()
print(f"Query made is {get_query()[-1]}")

<a id="3.g"></a>
### latest

Django Documentation - https://docs.djangoproject.com/en/4.2/ref/models/querysets/#latest

In [ ]:
Blog.objects.latest("id", "creation_date")

In [ ]:
get_query()

<a id="3.h"></a>
### earliest

Django Documentation - https://docs.djangoproject.com/en/4.2/ref/models/querysets/#earliest

In [ ]:
Blog.objects.earliest("id")
print(f"Query made is {get_query()[-1]}")

<a id="3.i"></a>
### last

Django Documentation - https://docs.djangoproject.com/en/4.2/ref/models/querysets/#last

In [ ]:
Blog.objects.last()
print(f"Query made is {get_query()[-1]}")

<a id="3.j"></a>
### first

Django Documentation - https://docs.djangoproject.com/en/4.2/ref/models/querysets/#first

In [ ]:
Blog.objects.first()
print(f"Query made is {get_query()[-1]}")

<a id="3.k"></a>
### exists

Django Documentation - https://docs.djangoproject.com/en/4.2/ref/models/querysets/#exists

In [ ]:
objs = Blog.objects.filter(name="Blog number 3")

In [ ]:
# Not optimized
if objs:
    print(f"Query made is {get_query()[-1]}")

In [ ]:
# optimized
if objs.exists():
    print(f"Query made is {get_query()[-1]}")

<a id="3.l"></a>
### contains

Django Documentation - https://docs.djangoproject.com/en/4.2/ref/models/querysets/#contains

In [ ]:
objs = Blog.objects.filter(name="Blog number 3")

In [ ]:
# Not optimized
if objs[0] in objs:
    print(f"Query made is {get_query()[-1]}")

In [ ]:
#Optimized
if objs.contains(objs[0]):
    print(f"Query made is {get_query()[-1]}")

<a id="3.m"></a>
### delete

Django Documentation - https://docs.djangoproject.com/en/4.2/ref/models/querysets/#delete

In [ ]:
Blog.objects.filter(name="Blog number 3").delete()

<a id="3.n"></a>
### update
<pre>
<b>Note</b>
 - The model’s save() method will not be called, and the pre_save and post_save signals will not be sent.
</pre>

Django Documentation - https://docs.djangoproject.com/en/4.2/ref/models/querysets/#update

In [ ]:
# Not optimized
obj = Blog.objects.get(id=1)
obj.name = "Somenameeee"
obj.save()

print(f"Query made is {get_query()[-1]}")

In [ ]:
# Optimized because no need to bring in memory (more will learn in future video)
Blog.objects.filter(id=1).update(name="Donnnnn")
print(f"Query made is {get_query()[-1]}")

<a id="3.o"></a>
### aggregate
<pre> Operation happen in Database directly</pre>

Django Documentation - https://docs.djangoproject.com/en/4.2/ref/models/querysets/#aggregate

In [ ]:
reset_queries()

In [ ]:
get_query()

In [ ]:
from django.db.models import Count
Blog.objects.aggregate(number_of_entries = Count("author"))
print(f"Query made is {get_query()[-1]}")

In [ ]:
Blog.objects.aggregate(Count("author"))
print(f"Query made is {get_query()[-1]}")

There are more aggregation Function like Count - https://docs.djangoproject.com/en/4.2/ref/models/querysets/#id6

## Hey If You are liked the content Do subscribe to my Youtube Channel
https://www.youtube.com/channel/UCpuKtSx_Cu8_ikyhlKAWBYA?sub_confirmation=1